# 載入套件

In [131]:
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
import json
import datetime

# 參數設定

In [132]:
# 欲下載新聞的股票關鍵字清單
list_0050 = '鴻海	, 聯發科	, 台達電	, 聯電	, 中華電	, 南亞	, 富邦金	, 中信金	, 中鋼	, 兆豐金	, 玉山金	, 統一	, 台塑	, 國泰金	, 日月光投控	, 第一金	, 合庫金	, 台化	, 元大金	, 中租-KY	, 華南金	, 台泥	, 廣達	, 開發金	, 和泰車	, 大立光	, 永豐金	, 華碩	, 台灣大	, 統一超	, 上海商銀	, 台新金	, 欣興	, 聯詠	, 長榮	, 國巨	, 臺股期貨	, 友達	, 矽力*-KY	, 研華	, 瑞昱	, 亞德客-KY	, 彰銀	, 遠傳	, 陽明	, 台塑化	, 萬海	, 豐泰	, 南亞科	, 南電	,'
list_0051 = "智邦\
,華新\
,光寶科\
,遠東新\
,可成\
,和碩\
,新光金\
,亞泥\
,群創\
,仁寶\
,旭隼\
,力積電\
,儒鴻\
,緯創\
,微星\
聯強,\
大同,\
長榮航,\
致茂,\
英業達,\
緯穎,\
大聯大,\
臻鼎-KY,\
臺企銀,\
正新,\
宏碁,\
巨大,\
世芯-KY,\
大成鋼,\
台灣高鐵,\
華邦電,\
旺宏,\
華航,\
嘉澤,\
信邦,\
東元,\
力成,\
鴻準,\
上銀,\
興富發,\
群光,\
華通,\
創意,\
台光電,\
健鼎,\
瑞儀,\
美利達,\
台中銀,\
技嘉,\
潤泰新\
台肥,\
宏達電,\
寶成,\
貿聯-KY,\
聚陽,\
佳世達,\
華新科,\
健策,\
聯邦銀,\
億豐,\
國票金,\
京元電子,\
裕融,\
北極星藥業-KY,\
京城銀,\
大成,\
永豐餘,\
長興,\
東和鋼鐵,\
川湖,\
遠東銀,\
玉晶光,\
智原,\
台玻,\
金像電,\
中保科,\
富邦媒,\
福懋,\
同欣電,\
富采,\
祥碩,\
景碩,\
慧洋-KY,\
文曄,\
采鈺,\
愛普,\
裕隆,\
潤泰全,\
中鴻,\
新唐,\
AES-KY,\
遠雄,\
和潤企業,\
裕民,\
美時,\
超豐,\
台勝科,\
裕日車,\
燁輝"

In [133]:
search_0050 = list_0050.strip('\t,').split('\t, ')
search_0051 = list_0051.strip(',').split(',')
QueryList = search_0050 + search_0051

QueryList.append('台灣50')
QueryList.append('0050')
QueryList.append('0051')
# 新聞下載起始日
StartDate = (datetime.date.today() + datetime.timedelta(days=-7)).strftime('%Y-%m-%d')
StartDate

'2022-11-09'

In [134]:
print(datetime.datetime.now())

2022-11-16 21:45:23.696293


In [135]:
searchList_all = []
sz = len(QueryList)
terms = int(sz/20)+1
for i in range(terms):
    searchList_all.append(QueryList[20*i:20*(i+1)])

# 相關函數

In [136]:
# 整理Google新聞資料用
def arrangeGoogleNews(elem):
    return ([elem.find('link').getText(),
             elem.find('pubDate').getText(),
            elem.find('source').getText()])

In [137]:
# 擷取各家新聞網站新聞函數
def beautifulSoupNews(url): # url為transfer後的(不是google news上的)
    global image_count
    # 設定hearers
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/87.0.4280.141 Safari/537.36'}

    # 下載網站原始碼
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        transfer_url = soup.find("meta", attrs={"property": "og:url"})['content']
        # 判斷url網域做對應文章擷取
        domain = re.findall('https://[^/]*', transfer_url)[0].replace('https://', '')
    except: 
        domain = "unknow"

    if domain == 'udn.com': # 聯合新聞網
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('figure', class_='article-content__cover')[0] 
            media = str(item.picture.source)
            start_idx = media.find('srcset')+ 8
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]

            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except: 
            pass
        
    elif domain in ['tw.stock.yahoo.com', 'tw.news.yahoo.com']:# Yahoo奇摩
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('img', class_='caas-img caas-lazy has-preview')[0] 
            media = str(item)
            start_idx = media.find('data-src')+ 10
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]
            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except:
            pass
        
    elif domain == 'money.udn.com':# 經濟日報
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('img', class_='lazyload')[0] 
            media = str(item)
            start_idx = media.find('src')+ 5
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]
            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except:
            pass
        
    elif domain == 'www.chinatimes.com':# 中時新聞網
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('img', class_='photo')[0] 
            media = str(item)
            start_idx = media.find('src')+ 5
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]
            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except:
            pass
    elif domain == 'ctee.com.tw':# 工商時報
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('img', alt="")[0] 
            media = str(item)
            start_idx = media.find('data-src')+ 10
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]
            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except:
            pass
    elif domain == 'news.cnyes.com': # 鉅亨網
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('img')[8]
            media = str(item)
            start_idx = media.find('src')+ 5
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]
            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except:
            pass
    elif domain == 'finance.ettoday.net':# ETtoday
        try: # some news do not have pic.
            # get image url
            item = soup.find_all('img')[1]
            media = str(item)
            start_idx = media.find('src')+5
            media[start_idx:]
            intermedia = media[start_idx:]
            end_idx = intermedia.find('"')
            img_url = intermedia[:end_idx]
            # download image
            response = requests.get(img_url)
            if response.status_code:
                fp = open(f'image/{image_count}.png', 'wb')
                image_count += 1
                fp.write(response.content)
                fp.close()
        except:
            pass

    return

# Add new website sources

In [138]:
# url='https://finance.ettoday.net/news/2376967'

In [139]:
# headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
#                              'Chrome/87.0.4280.141 Safari/537.36'}
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# transfer_url = soup.find("meta", attrs={"property": "og:url"})['content']
# # 判斷url網域做對應文章擷取
# domain = re.findall('https://[^/]*', transfer_url)[0].replace('https://', '')
# domain

In [140]:
# # item = soup.find_all('img', class_='photo')[0] 
# item = soup.find_all('img')[1]
# media = str(item)
# start_idx = media.find('src')+5
# media[start_idx:]
# intermedia = media[start_idx:]
# end_idx = intermedia.find('"')
# img_url = intermedia[:end_idx]
# img_url

# 迴圈下載股票清單的Google新聞圖片

In [144]:
searchList = searchList_all[0]
image_count=1

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

目前正在搜尋股票: 鴻海 在Google的新聞清單  進度: 1 / 20
目前正在下載: 鴻海 各家新聞  進度: 1 / 9
目前正在下載: 鴻海 各家新聞  進度: 2 / 9
目前正在下載: 鴻海 各家新聞  進度: 3 / 9
目前正在下載: 鴻海 各家新聞  進度: 4 / 9
目前正在下載: 鴻海 各家新聞  進度: 5 / 9
目前正在下載: 鴻海 各家新聞  進度: 6 / 9
目前正在下載: 鴻海 各家新聞  進度: 7 / 9
目前正在下載: 鴻海 各家新聞  進度: 8 / 9
目前正在下載: 鴻海 各家新聞  進度: 9 / 9
目前正在搜尋股票: 聯發科 在Google的新聞清單  進度: 2 / 20
目前正在下載: 聯發科 各家新聞  進度: 1 / 28
目前正在下載: 聯發科 各家新聞  進度: 2 / 28
目前正在下載: 聯發科 各家新聞  進度: 3 / 28
目前正在下載: 聯發科 各家新聞  進度: 4 / 28
目前正在下載: 聯發科 各家新聞  進度: 5 / 28
目前正在下載: 聯發科 各家新聞  進度: 6 / 28
目前正在下載: 聯發科 各家新聞  進度: 7 / 28
目前正在下載: 聯發科 各家新聞  進度: 8 / 28
目前正在下載: 聯發科 各家新聞  進度: 9 / 28
目前正在下載: 聯發科 各家新聞  進度: 10 / 28
目前正在下載: 聯發科 各家新聞  進度: 11 / 28
目前正在下載: 聯發科 各家新聞  進度: 12 / 28
目前正在下載: 聯發科 各家新聞  進度: 13 / 28
目前正在下載: 聯發科 各家新聞  進度: 14 / 28
目前正在下載: 聯發科 各家新聞  進度: 15 / 28
目前正在下載: 聯發科 各家新聞  進度: 16 / 28
目前正在下載: 聯發科 各家新聞  進度: 17 / 28
目前正在下載: 聯發科 各家新聞  進度: 18 / 28
目前正在下載: 聯發科 各家新聞  進度: 19 / 28
目前正在下載: 聯發科 各家新聞  進度: 20 / 28
目前正在下載: 聯發科 各家新聞  進度: 21 / 28
目前正在下載: 聯發科 各家新聞  進度: 22 / 28
目前正在下載: 聯發科 各家新聞  進度: 23 / 28

目前正在下載: 兆豐金 各家新聞  進度: 14 / 17
目前正在下載: 兆豐金 各家新聞  進度: 15 / 17
目前正在下載: 兆豐金 各家新聞  進度: 16 / 17
目前正在下載: 兆豐金 各家新聞  進度: 17 / 17
目前正在搜尋股票: 玉山金 在Google的新聞清單  進度: 11 / 20
目前正在下載: 玉山金 各家新聞  進度: 1 / 28
目前正在下載: 玉山金 各家新聞  進度: 2 / 28
目前正在下載: 玉山金 各家新聞  進度: 3 / 28
目前正在下載: 玉山金 各家新聞  進度: 4 / 28
目前正在下載: 玉山金 各家新聞  進度: 5 / 28
目前正在下載: 玉山金 各家新聞  進度: 6 / 28
目前正在下載: 玉山金 各家新聞  進度: 7 / 28
目前正在下載: 玉山金 各家新聞  進度: 8 / 28
目前正在下載: 玉山金 各家新聞  進度: 9 / 28
目前正在下載: 玉山金 各家新聞  進度: 10 / 28
目前正在下載: 玉山金 各家新聞  進度: 11 / 28
目前正在下載: 玉山金 各家新聞  進度: 12 / 28
目前正在下載: 玉山金 各家新聞  進度: 13 / 28
目前正在下載: 玉山金 各家新聞  進度: 14 / 28
目前正在下載: 玉山金 各家新聞  進度: 15 / 28
目前正在下載: 玉山金 各家新聞  進度: 16 / 28
目前正在下載: 玉山金 各家新聞  進度: 17 / 28
目前正在下載: 玉山金 各家新聞  進度: 18 / 28
目前正在下載: 玉山金 各家新聞  進度: 19 / 28
目前正在下載: 玉山金 各家新聞  進度: 20 / 28
目前正在下載: 玉山金 各家新聞  進度: 21 / 28
目前正在下載: 玉山金 各家新聞  進度: 22 / 28
目前正在下載: 玉山金 各家新聞  進度: 23 / 28
目前正在下載: 玉山金 各家新聞  進度: 24 / 28
目前正在下載: 玉山金 各家新聞  進度: 25 / 28
目前正在下載: 玉山金 各家新聞  進度: 26 / 28
目前正在下載: 玉山金 各家新聞  進度: 27 / 28
目前正在下載: 玉山金 各家新聞  進度: 28 / 28
目前正在搜尋股票:

目前正在下載: 台化 各家新聞  進度: 22 / 35
目前正在下載: 台化 各家新聞  進度: 23 / 35
目前正在下載: 台化 各家新聞  進度: 24 / 35
目前正在下載: 台化 各家新聞  進度: 25 / 35
目前正在下載: 台化 各家新聞  進度: 26 / 35
目前正在下載: 台化 各家新聞  進度: 27 / 35
目前正在下載: 台化 各家新聞  進度: 28 / 35
目前正在下載: 台化 各家新聞  進度: 29 / 35
目前正在下載: 台化 各家新聞  進度: 30 / 35
目前正在下載: 台化 各家新聞  進度: 31 / 35
目前正在下載: 台化 各家新聞  進度: 32 / 35
目前正在下載: 台化 各家新聞  進度: 33 / 35
目前正在下載: 台化 各家新聞  進度: 34 / 35
目前正在下載: 台化 各家新聞  進度: 35 / 35
目前正在搜尋股票: 元大金 在Google的新聞清單  進度: 19 / 20
目前正在下載: 元大金 各家新聞  進度: 1 / 25
目前正在下載: 元大金 各家新聞  進度: 2 / 25
目前正在下載: 元大金 各家新聞  進度: 3 / 25
目前正在下載: 元大金 各家新聞  進度: 4 / 25
目前正在下載: 元大金 各家新聞  進度: 5 / 25
目前正在下載: 元大金 各家新聞  進度: 6 / 25
目前正在下載: 元大金 各家新聞  進度: 7 / 25
目前正在下載: 元大金 各家新聞  進度: 8 / 25
目前正在下載: 元大金 各家新聞  進度: 9 / 25
目前正在下載: 元大金 各家新聞  進度: 10 / 25
目前正在下載: 元大金 各家新聞  進度: 11 / 25
目前正在下載: 元大金 各家新聞  進度: 12 / 25
目前正在下載: 元大金 各家新聞  進度: 13 / 25
目前正在下載: 元大金 各家新聞  進度: 14 / 25
目前正在下載: 元大金 各家新聞  進度: 15 / 25
目前正在下載: 元大金 各家新聞  進度: 16 / 25
目前正在下載: 元大金 各家新聞  進度: 17 / 25
目前正在下載: 元大金 各家新聞  進度: 18 / 25
目前正在下載: 元大金 各家新聞  進度: 1

In [145]:
searchList = searchList_all[1]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

目前正在搜尋股票: 華南金 在Google的新聞清單  進度: 1 / 20
目前正在下載: 華南金 各家新聞  進度: 1 / 16
目前正在下載: 華南金 各家新聞  進度: 2 / 16
目前正在下載: 華南金 各家新聞  進度: 3 / 16
目前正在下載: 華南金 各家新聞  進度: 4 / 16
目前正在下載: 華南金 各家新聞  進度: 5 / 16
目前正在下載: 華南金 各家新聞  進度: 6 / 16
目前正在下載: 華南金 各家新聞  進度: 7 / 16
目前正在下載: 華南金 各家新聞  進度: 8 / 16
目前正在下載: 華南金 各家新聞  進度: 9 / 16
目前正在下載: 華南金 各家新聞  進度: 10 / 16
目前正在下載: 華南金 各家新聞  進度: 11 / 16
目前正在下載: 華南金 各家新聞  進度: 12 / 16
目前正在下載: 華南金 各家新聞  進度: 13 / 16
目前正在下載: 華南金 各家新聞  進度: 14 / 16
目前正在下載: 華南金 各家新聞  進度: 15 / 16
目前正在下載: 華南金 各家新聞  進度: 16 / 16
目前正在搜尋股票: 台泥 在Google的新聞清單  進度: 2 / 20
目前正在下載: 台泥 各家新聞  進度: 1 / 18
目前正在下載: 台泥 各家新聞  進度: 2 / 18
目前正在下載: 台泥 各家新聞  進度: 3 / 18
目前正在下載: 台泥 各家新聞  進度: 4 / 18
目前正在下載: 台泥 各家新聞  進度: 5 / 18
目前正在下載: 台泥 各家新聞  進度: 6 / 18
目前正在下載: 台泥 各家新聞  進度: 7 / 18
目前正在下載: 台泥 各家新聞  進度: 8 / 18
目前正在下載: 台泥 各家新聞  進度: 9 / 18
目前正在下載: 台泥 各家新聞  進度: 10 / 18
目前正在下載: 台泥 各家新聞  進度: 11 / 18
目前正在下載: 台泥 各家新聞  進度: 12 / 18
目前正在下載: 台泥 各家新聞  進度: 13 / 18
目前正在下載: 台泥 各家新聞  進度: 14 / 18
目前正在下載: 台泥 各家新聞  進度: 15 / 18
目前正在下載: 台泥 各家新聞  進度: 16 / 

目前正在下載: 統一超 各家新聞  進度: 21 / 38
目前正在下載: 統一超 各家新聞  進度: 22 / 38
目前正在下載: 統一超 各家新聞  進度: 23 / 38
目前正在下載: 統一超 各家新聞  進度: 24 / 38
目前正在下載: 統一超 各家新聞  進度: 25 / 38
目前正在下載: 統一超 各家新聞  進度: 26 / 38
目前正在下載: 統一超 各家新聞  進度: 27 / 38
目前正在下載: 統一超 各家新聞  進度: 28 / 38
目前正在下載: 統一超 各家新聞  進度: 29 / 38
目前正在下載: 統一超 各家新聞  進度: 30 / 38
目前正在下載: 統一超 各家新聞  進度: 31 / 38
目前正在下載: 統一超 各家新聞  進度: 32 / 38
目前正在下載: 統一超 各家新聞  進度: 33 / 38
目前正在下載: 統一超 各家新聞  進度: 34 / 38
目前正在下載: 統一超 各家新聞  進度: 35 / 38
目前正在下載: 統一超 各家新聞  進度: 36 / 38
目前正在下載: 統一超 各家新聞  進度: 37 / 38
目前正在下載: 統一超 各家新聞  進度: 38 / 38
目前正在搜尋股票: 上海商銀 在Google的新聞清單  進度: 11 / 20
目前正在下載: 上海商銀 各家新聞  進度: 1 / 14
目前正在下載: 上海商銀 各家新聞  進度: 2 / 14
目前正在下載: 上海商銀 各家新聞  進度: 3 / 14
目前正在下載: 上海商銀 各家新聞  進度: 4 / 14
目前正在下載: 上海商銀 各家新聞  進度: 5 / 14
目前正在下載: 上海商銀 各家新聞  進度: 6 / 14
目前正在下載: 上海商銀 各家新聞  進度: 7 / 14
目前正在下載: 上海商銀 各家新聞  進度: 8 / 14
目前正在下載: 上海商銀 各家新聞  進度: 9 / 14
目前正在下載: 上海商銀 各家新聞  進度: 10 / 14
目前正在下載: 上海商銀 各家新聞  進度: 11 / 14
目前正在下載: 上海商銀 各家新聞  進度: 12 / 14
目前正在下載: 上海商銀 各家新聞  進度: 13 / 14
目前正在下載: 上海商銀 各家新聞  進度: 14

ConnectionError: HTTPSConnectionPool(host='news.google.com', port=443): Max retries exceeded with url: /news/rss/search/section/q/%E6%AC%A3%E8%88%88/?hl=zh-tw&gl=TW&ned=zh-tw_tw (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020EB1F6C640>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
searchList = searchList_all[2]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

In [ ]:
searchList = searchList_all[3]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

In [ ]:
searchList = searchList_all[4]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

In [ ]:
searchList = searchList_all[5]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

In [ ]:
searchList = searchList_all[6]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])

In [ ]:
searchList = searchList_all[7]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]
    
    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['link', 'pub_date','source'])
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)
    
    # 迴圈爬取新聞圖片
    for iLink in range(len(df['link'])):
        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))
        beautifulSoupNews(url=df['link'][iLink])